In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer

from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

--2024-06-14 19:27:38--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  55.0MB/s    in 0.5s    

2024-06-14 19:27:39 (55.0 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
df_books.head()

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [ ]:
df_ratings.head()

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [ ]:
df_all = pd.merge(left=df_books, right=df_ratings, how="left", left_on='isbn', right_on='isbn')
df_all.head()

,isbn,title,author,user,rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2.0,0.0
1,0002005018,Clara Callan,Richard Bruce Wright,8.0,5.0
2,0002005018,Clara Callan,Richard Bruce Wright,11400.0,0.0
3,0002005018,Clara Callan,Richard Bruce Wright,11676.0,8.0
4,0002005018,Clara Callan,Richard Bruce Wright,41385.0,0.0


In [ ]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1032384 entries, 0 to 1032383
Data columns (total 5 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   isbn    1032384 non-null  object 
 1   title   1032384 non-null  object 
 2   author  1032382 non-null  object 
 3   user    1031175 non-null  float64
 4   rating  1031175 non-null  float32
dtypes: float32(1), float64(1), object(3)
memory usage: 35.4+ MB


In [ ]:
# To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.
usrs_count = df_all.groupby(df_all['user'])['rating'].count()
filtered_uses = usrs_count[usrs_count > 200].index

total_filtered_usrs = len(usrs_count) - len(filtered_uses)
print(f'# filtered {total_filtered_usrs} users')

df_all = df_all[df_all['user'].isin(filtered_uses)]
df_all.info()

# filtered 91296 users
<class 'pandas.core.frame.DataFrame'>
Index: 474020 entries, 3 to 1032376
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   isbn    474020 non-null  object 
 1   title   474020 non-null  object 
 2   author  474019 non-null  object 
 3   user    474020 non-null  float64
 4   rating  474020 non-null  float32
dtypes: float32(1), float64(1), object(3)
memory usage: 19.9+ MB


In [ ]:
books_count = df_all.groupby(df_all['title'])['rating'].count()
filtered_books = books_count[books_count > 100].index

total_filtered_books = len(books_count) - len(filtered_books)
print(f'# filtered {total_filtered_books} books')

df_all = df_all[df_all['title'].isin(filtered_books)]
df_all.info()

# filtered 0 books
<class 'pandas.core.frame.DataFrame'>
Index: 20899 entries, 31 to 1029984
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   isbn      20899 non-null  object 
 1   title     20899 non-null  object 
 2   author    20899 non-null  object 
 3   user      20899 non-null  float64
 4   rating    20899 non-null  float32
 5   len_name  20899 non-null  int64  
dtypes: float32(1), float64(1), int64(1), object(3)
memory usage: 1.0+ MB


In [ ]:
df_all = df_all.drop_duplicates(subset=['title'], keep='first').reset_index(drop=True)
#df_all.value_counts(subset=['title'])
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242154 entries, 0 to 242153
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   isbn    242154 non-null  object 
 1   title   242154 non-null  object 
 2   author  242152 non-null  object 
 3   user    241080 non-null  float64
 4   rating  241080 non-null  float32
dtypes: float32(1), float64(1), object(3)
memory usage: 8.3+ MB


In [ ]:
vectorizer = TfidfVectorizer()
input_data_encoded_titles = vectorizer.fit_transform(df_all['title'])

list_titles = list(df_all['title'])
titles_vector_dict = {ind: title  for ind, title in enumerate(list_titles)}

input_data_encoded_titles.shape

(242154, 78768)

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  num_neighbors = 5 + 1 #the number of predictions the model will return plus the book itself
  knn_model = NearestNeighbors(n_neighbors=5)
  knn_model.fit(input_data_encoded_titles)

  recommended_books = [book, []]

  encoded_book = vectorizer.transform([book])
  distances, books_recomended = knn_model.kneighbors(encoded_book, n_neighbors=num_neighbors, return_distance=True)

  for ind, book_ind in enumerate(books_recomended[0]):
    book_title = list_titles[book_ind]
    book_distance = distances[0][ind]
    if book_title != book:
      recommended_books[1].append([book_title, book_distance])

  return recommended_books

In [ ]:
get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")

["Where the Heart Is (Oprah's Book Club (Paperback))",
 [["The Heart Is a Lonely Hunter (Oprah's Book Club)", 0.7869814761491492],
  ["The Heart of a Woman (Oprah's Book Club (Hardcover))", 0.8295409111242987],
  ["The Book of Ruth (Oprah's Book Club (Paperback))", 0.8504510850581553],
  ['Where the Heart Is', 0.8564939963490482],
  ["Open House (Oprah's Book Club (Paperback))", 0.8675346194302506]]]

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["The Heart Is a Lonely Hunter (Oprah's Book Club)", 0.7869814761491492], ["The Heart of a Woman (Oprah's Book Club (Hardcover))", 0.8295409111242987], ["The Book of Ruth (Oprah's Book Club (Paperback))", 0.8504510850581553], ['Where the Heart Is', 0.8564939963490482], ["Open House (Oprah's Book Club (Paperback))", 0.8675346194302506]]]
You haven't passed yet. Keep trying!
